# Notebook: Compare Model Performance

In [296]:
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import levene
import statsmodels.api as sm
from scipy import stats
import numpy as np
import json
from scipy.stats import wilcoxon

In [297]:
task_name = "end_2_end_absa"
metric = "eval_f1_macro"
side = "greater"

n_real_a = "500"
n_synth_a = "1500"
model_a = "GPT-3" # only_real
cond_a = "fixed" if n_real_a == "25" else "random"

n_real_b = "500"
n_synth_b = "0"
model_b = "only_real"
cond_b = "fixed" if n_real_b == "25" else "random"

In [298]:
json_path_only_real = f"../07 train models/results_json/results_{model_a}_real{n_real_a}_synth{n_synth_a}_{task_name}_{cond_a}.json"
json_path_with_synth = f"../07 train models/results_json/results_{model_b}_real{n_real_b}_synth{n_synth_b}_{task_name}_{cond_b}.json"

with open(json_path_only_real, 'r') as json_file:
     results_only_real = json.load(json_file)

with open(json_path_with_synth, 'r') as json_file:
     results_with_synth = json.load(json_file)

In [299]:
only_real_f1 = [res[metric]  for res in results_only_real["single_split_results"]]
with_synth_f1 = [res[metric]  for res in results_with_synth["single_split_results"]]

In [300]:
only_real_f1

[0.6788658141517477,
 0.6903200946797677,
 0.7498235770196375,
 0.7215266429599065,
 0.6991554215121627,
 0.7605027156751295]

In [301]:
with_synth_f1

[0.7386960249248972,
 0.686365310059827,
 0.7294402257026843,
 0.6053405025602924,
 0.686424506909682,
 0.7579540768029647]

## Normalverteilung

In [302]:
differences = np.array(only_real_f1) - np.array(with_synth_f1)
statistic, p_value_normal = shapiro(differences)

print(f"Shapiro-Wilk-Test: Statistik = {statistic}, p = {p_value_normal}")

alpha_normal = 0.05
if p_value_normal >= alpha_normal:
    print("Die Differenzen könnten normalverteilt sein (nicht signifikant)")
else:
    print("Die Differenzen sind möglicherweise nicht normalverteilt (signifikant)")


Shapiro-Wilk-Test: Statistik = 0.8765276670455933, p = 0.2534877359867096
Die Differenzen könnten normalverteilt sein (nicht signifikant)


## Test for Normal Distribution Scenario

In [303]:
from scipy.stats import ttest_rel, wilcoxon

if p_value_normal >= alpha_normal:
    # Gepaarter t-Test (einseitig)
    t_statistic, p_value = ttest_rel(only_real_f1, with_synth_f1, alternative=side)
    # alternative kann 'greater' oder 'less' sein, je nachdem, welche Seite Sie testen möchten

    # Ausgabe der Ergebnisse
    print(f"Gepaarter t-Test (einseitig): t = {t_statistic}, p = {p_value}")
else:
    # Wilcoxon Signed-Rank Test (einseitig)
    statistic, p_value = wilcoxon(only_real_f1, with_synth_f1, alternative=side)
    # alternative kann 'greater' oder 'less' sein, je nachdem, welche Seite Sie testen möchten

    # Ausgabe der Ergebnisse
    print(f"Wilcoxon Signed-Rank Test (einseitig): statistic = {statistic}, p = {p_value}")


Gepaarter t-Test (einseitig): t = 0.689546571841807, p = 0.2605841253364344


In [304]:
0.05 / 3

0.016666666666666666

In [316]:
from statsmodels.stats import multitest

# Liste der p-Werte
p_values = [0.015625, 0.0035205120720581454, 0.03125, 0.15915229571594033,
            0.031241494687033116, 0.0018269932163866459, 0.0004437415162184869, 0.00015110222444678515]

# Bonferroni-Holm-Korrektur
reject, corrected_p_values, _, _ = multitest.multipletests(
    p_values, method='holm')

# Ausgabe der p-Werte und Signifikanz
for i in range(len(p_values)):
    print(
        f'Ursprünglicher p-Wert: {p_values[i]:.6f}, Korrigierter p-Wert: {corrected_p_values[i]:.6f}, Signifikant: {"Ja" if reject[i] else "Nein"}')

Ursprünglicher p-Wert: 0.015625, Korrigierter p-Wert: 0.062500, Signifikant: Nein
Ursprünglicher p-Wert: 0.003521, Korrigierter p-Wert: 0.017603, Signifikant: Ja
Ursprünglicher p-Wert: 0.031250, Korrigierter p-Wert: 0.093724, Signifikant: Nein
Ursprünglicher p-Wert: 0.159152, Korrigierter p-Wert: 0.159152, Signifikant: Nein
Ursprünglicher p-Wert: 0.031241, Korrigierter p-Wert: 0.093724, Signifikant: Nein
Ursprünglicher p-Wert: 0.001827, Korrigierter p-Wert: 0.010962, Signifikant: Ja
Ursprünglicher p-Wert: 0.000444, Korrigierter p-Wert: 0.003106, Signifikant: Ja
Ursprünglicher p-Wert: 0.000151, Korrigierter p-Wert: 0.001209, Signifikant: Ja


In [317]:
from statsmodels.stats import multitest

# Liste der p-Werte
p_values = [0.10689158577809782, 0.09538977852456092, 0.5703838309041978]

# Bonferroni-Holm-Korrektur
reject, corrected_p_values, _, _ = multitest.multipletests(
    p_values, method='holm')

# Ausgabe der p-Werte und Signifikanz
for i in range(len(p_values)):
    print(
        f'Ursprünglicher p-Wert: {p_values[i]:.6f}, Korrigierter p-Wert: {corrected_p_values[i]:.6f}, Signifikant: {"Ja" if reject[i] else "Nein"}')

Ursprünglicher p-Wert: 0.106892, Korrigierter p-Wert: 0.286169, Signifikant: Nein
Ursprünglicher p-Wert: 0.095390, Korrigierter p-Wert: 0.286169, Signifikant: Nein
Ursprünglicher p-Wert: 0.570384, Korrigierter p-Wert: 0.570384, Signifikant: Nein


In [319]:
from statsmodels.stats import multitest

# Liste der p-Werte
p_values = [0.01966409807826196, 0.17519189198839413, 0.2605841253364344]

# Bonferroni-Holm-Korrektur
reject, corrected_p_values, _, _ = multitest.multipletests(
    p_values, method='holm')

# Ausgabe der p-Werte und Signifikanz
for i in range(len(p_values)):
    print(
        f'Ursprünglicher p-Wert: {p_values[i]:.6f}, Korrigierter p-Wert: {corrected_p_values[i]:.6f}, Signifikant: {"Ja" if reject[i] else "Nein"}')

Ursprünglicher p-Wert: 0.019664, Korrigierter p-Wert: 0.058992, Signifikant: Nein
Ursprünglicher p-Wert: 0.175192, Korrigierter p-Wert: 0.350384, Signifikant: Nein
Ursprünglicher p-Wert: 0.260584, Korrigierter p-Wert: 0.350384, Signifikant: Nein
